# Advent of Code 2020
Basic imports of stuff

In [1]:
from aocd import submit
from aocd.models import Puzzle, User
from pathlib import Path
from itertools import islice, combinations
from functools import reduce
from collections import deque
import re

Useful stuff

In [2]:
def first(iterable):
    '''returns the first item of an iterable'''
    return next(iter(iterable))

def nth(iterable, n, default=None):
    '''Returns the nth item in an iterable or a default value'''
    return next(islice(iterable, n, None), default)

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def tail(n, iterable):
    "Return an iterator over the last n items"
    # tail(3, 'ABCDEFG') --> E F G
    return iter(deque(iterable, maxlen=n))

def prod(numbers):
    '''Returns the product of all the numbers'''
    return reduce((lambda x,y: x*y), numbers)


## Day 01
### Part 1
The puzzle input is a list of numbers (provided as a long string, separated by newlines).
I need to fine the 2 entries that sum to 2020, then provide the multiple of the 2 entries


In [3]:
p = Puzzle(year=2020, day=1)
d = [int(x) for x in p.input_data.split()]

In [4]:
result = None
for entries in combinations(d,2):
    if 2020 == sum(entries):
        result = prod(entries)
        break

if result:
    print(result)
    p.answer_a = result

960075


### Part 2
Same puzzle but find the combination of 3 entries that add up to 2020

In [5]:
result = None
for entries in combinations(d,3):
    if 2020 == sum(entries):
        result = prod(entries)
        break

if result:
    print(result)
    p.answer_b = result

212900130


## Day 02
### Part 1
Validate passwords in a list (puzzle input) using rules associated with each password.  Using regex to make this work.


In [6]:
p = Puzzle(year=2020, day=2)


In [7]:
result = 0

parser = re.compile('(\d*)-(\d*).(\w): (\w*)')

for lo, hi, letter, password in parser.findall(p.input_data):
    if int(lo) <= password.count(letter) <= int(hi):
        result+=1

print(result)
p.answer_a = result

422


### Part 2
This time the numbers tell you which indexes to check.  Only one of the indexes can contain the specified letter

In [8]:
result = 0

parser = re.compile('(\d*)-(\d*).(\w): (\w*)')

for lo, hi, letter, password in parser.findall(p.input_data):
    if int(lo) < int(hi) <= len(password):
        if (password[int(lo)-1] == letter) ^ (password[int(hi)-1] == letter):
            result+=1

print(result)
p.answer_b = result

451


## Day 3
### Part 1


In [10]:
p = Puzzle(day=3,year=2020)

In [14]:
d = p.input_data.splitlines()

In [33]:
result = 0

start = (0,0)
slope = (3,1)
ncol = len(d[0])

here = start
while here[1] < len(d):
    if d[here[1]][here[0]] == '#':
        result += 1
    here = (here[0] + slope[0]) % ncol , here[1] + slope[1]

p.answer_a= result
 

That's the right answer!  You are one gold star closer to saving your vacation. [Continue to Part Two]


### Part 2

In [35]:
results = []

start = (0,0)
ncol = len(d[0])

slopes = [(1,1), (3,1), (5,1), (7,1), (1,2)]

for slope in slopes:
    here = start
    result = 0
    while here[1] < len(d):
        if d[here[1]][here[0]] == '#':
            result += 1
        here = (here[0] + slope[0]) % ncol , here[1] + slope[1]
    
    results.append(result)

p.answer_b = prod(results)

That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 3! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


## Day 4
### Part 1

In [54]:
p = Puzzle(day=4, year=2020)

In [62]:

parser = re.compile('(\w{3}):(\S+)')

def has_all_fields(passport):
    required_fields = set(['byr','iyr','eyr','hgt','hcl','ecl','pid'])
    detected_fields = set(field[0] for field in parser.findall(passport))
    return not(required_fields - detected_fields)

passports = p.input_data.split('\n\n')
valid_passports = [passport for passport in passports if has_all_fields(passport)]

p.answer_a = len(valid_passports)

239
